Global Constants

In [1]:
import codecs, re, sys

sys.path.append("~/Documents/GitHub/Voices-Now-SRT-Generator/Scripts")

from ProTools.Timecode import Timecode, OffsetType

from Captions.CaptionMaker import CaptionMaker
from Captions.LanguageDatabase import LanguageDatabase

LANGUAGE_MANAGER = LanguageDatabase()

PROJECT_PATH = "/Volumes/VONGeneral/LDS/~Current Projects/MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts/{0}/{1}/MFD_TrainingCouncil{2}.{3}"

def split_languages(timecode_offset: Timecode = None, timecode_offset_type: OffsetType = OffsetType.NONE, languages = []):
    for lang_code in languages:
        language = LANGUAGE_MANAGER.get_language(lang_code)
        name = language.name
        max_char_count = LANGUAGE_MANAGER.get_max_line_count(lang_code)
        split = LANGUAGE_MANAGER.language_supports_split(lang_code)

        combined_srt = PROJECT_PATH.format(name, "Initial SRTs", "Combined", "srt")

        with codecs.open(combined_srt, "w+", encoding="utf-8") as combined_file:
            srt_offset = 0

            for i in range(1, 4):
                script_name = PROJECT_PATH.format(name, "Scripts", "0" + str(i), "docx")
                edl_name = PROJECT_PATH.format("~Template", "EDLs", "0" + str(i), "txt")
                srt_name = PROJECT_PATH.format(name, "Initial SRTs", "0" + str(i), "srt")                

                print(script_name)

                srt_maker = CaptionMaker(script_name, edl_name, "EDL", lang_code, srt_name, max_char_count, split)
                srt_offset = srt_maker.create_captions(timecode_offset, timecode_offset_type, srt_offset)

                with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
                    combined_file.write(srt_file.read())

def no_split_languages(timecode_offset: tuple = None, languages = []):
    for lang_code in languages:
        full_lang = LANGUAGE_MANAGER.get_lang(lang_code)

        combined_srt = PROJECT_PATH.format(full_lang, "Initial SRTs", "Combined", "srt")

        with codecs.open(combined_srt, "w+", encoding="utf-8") as combined_file:
            srt_offset = 0

            timecode_offsets = [None, ('dly', Timecode.from_total_frames("00:04:33;24")), ('dly', Timecode.from_total_frames("00:08:16;11"))]

            for i in range(1, 4):
                script_name = PROJECT_PATH.format(full_lang, "Scripts", "0" + str(i), "docx")
                # edl_name = project_path.format("~Template", "EDLs", "0" + str(i), "txt")
                srt_name = PROJECT_PATH.format(full_lang, "Initial SRTs", "0" + str(i), "srt")

                print(script_name)

                srt_maker = CaptionMaker(script_name, None, "MRK", lang_code, srt_name, 44, False)
                srt_offset = srt_maker.create_captions(timecode_offsets[i-1], srt_offset)

                with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
                    combined_file.write(srt_file.read())

def offset_single_srt(srt_name: str, offset: Timecode):
    pattern = r"\d{2}:\d{2}:\d{2},\d{3}"

    with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
        srt_data = srt_file.read()

    offset_srt_data = re.sub(pattern, lambda match: (Timecode.from_total_frames(match.group()) - offset).get_timecode_in_ms(), srt_data)

    with codecs.open(srt_name, "w", encoding="utf-8") as srt_file:
        srt_file.write(offset_srt_data)

def offset_srts(offset: Timecode, languages: list = []):
    for lang_code in languages:
        full_lang = LANGUAGE_MANAGER.get_lang(lang_code)

        for i in range(1, 4):
            srt_name = PROJECT_PATH.format(full_lang, "Initial SRTs", "0" + str(i), "srt")
            offset_single_srt(srt_name, offset)
        
        offset_single_srt(PROJECT_PATH.format(full_lang, "Initial SRTs", "Combined", "srt"), offset)
            
def specific_language(lang_code: str, timecode_offset: tuple = None):
    full_lang = LANGUAGE_MANAGER.get_lang(lang_code)

    combined_srt = PROJECT_PATH.format(full_lang, "Initial SRTs", "Combined", "srt")

    with codecs.open(combined_srt, "w+", encoding="utf-8") as combined_file:
        srt_offset = 0

        for i in range(1, 4):
            script_name = PROJECT_PATH.format(full_lang, "Scripts", "0" + str(i), "docx")
            edl_name = PROJECT_PATH.format(full_lang, "EDLs", "0" + str(i), "txt")
            srt_name = PROJECT_PATH.format(full_lang, "Initial SRTs", "0" + str(i), "srt")
            split = LANGUAGE_MANAGER.language_supports_split(lang_code)

            print(script_name)

            srt_maker = CaptionMaker(script_name, edl_name, "EDL", lang_code, srt_name, 44, split)
            srt_offset = srt_maker.create_captions(timecode_offset, srt_offset)

            with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
                combined_file.write(srt_file.read())

/Users/studiod/Documents/GitHub/Voices-Now-SRT-Generator/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:


languages = ["FRA"] # ["CEB", "FRA", "DEU", "ITA", "KHM", "KOR", "MLG", "MAH", "POR", "RUS", "SMO", "SPA", "TGL", "THA", "TON", "UKR", "VIE"]
no_split = ["JPN", "CMN"]

weird_placement = Timecode.from_string("00:00:14;25")

# offsets = [Timecode.from_frames("00:00:00;00"),
#            Timecode.from_frames("00:04:33;24"),
#            Timecode.from_frames("00:08:16;11")]

# specific_language("RUS", ("adv", weird_placement))
split_languages(weird_placement, OffsetType.ADVANCE, languages)



AttributeError: 'LanguageDatabase' object has no attribute 'get_lang'